::: {.callout-note appearance="simple"}
### Licensing Notice
Text and media: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)  
Code and snippets: [Apache License 2.0](http://www.apache.org/licenses/LICENSE-2.0)
:::

# RAG Fundamentals

**R**etrieval **A**ugmented **G**eneration